In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
import os

folder_path = "/home/v1xjian2/BDL/Bayesian_DL/datasets/aptos2019-blindness-detection/train_images" 

files = os.listdir(folder_path)

print("Num of files under this folder is: %d " % len(files))

In [ ]:
aptos2019_train_df = pd.read_csv('/home/v1xjian2/BDL/Bayesian_DL/datasets/aptos2019-blindness-detection/train.csv')

In [ ]:
aptos2019_train_df['diagnosis'].value_counts()

# Binarize targets

In [ ]:
aptos2019_train_df['target'] = (aptos2019_train_df['diagnosis'] >=2)*1

In [ ]:
aptos2019_train_df['target'].value_counts()

In [ ]:
aptos2019_train_df.to_csv('./binarised_data.csv')

# Split train and test

In [ ]:
'''
target_0 = aptos2019_train_df[aptos2019_train_df["target"] == 0]
target_1 = aptos2019_train_df[aptos2019_train_df["target"] == 1]

test_data_propotion = 0.3

n_0 = int(test_data_propotion * len(target_0))
n_1 = int(test_data_propotion * len(target_1))

sample_0 = target_0.sample(n=n_0, replace=False, random_state=0)
sample_1 = target_1.sample(n=n_1, replace=False, random_state=1)

test_split_df = pd.concat([sample_0, sample_1])
'''

In [ ]:
'''
test_split_df.to_csv('./test_split.csv')
'''

In [ ]:
'''
train_split_df = pd.merge(aptos2019_train_df, test_split_df, how="outer", indicator=True)
train_split_df = train_split_df[train_split_df["_merge"] == "left_only"]
train_split_df = train_split_df.drop(columns=["_merge"])
'''

In [ ]:
# train_split_df.to_csv('./train_split.csv')

# Have a look at these images

In [ ]:
from PIL import Image

image = Image.open('/home/v1xjian2/BDL/Bayesian_DL/datasets/aptos2019-blindness-detection/train_images/0a4e1a29ffff.png')
width, height = image.size
print(f'Original image size: {width} x {height}')
image = image.resize((256, 256), resample=Image.BILINEAR)
width, height = image.size
print(f'Image size after resize: {width} x {height}')

# Some code for fun

In [ ]:

"""
from RetinopathyDataset import RetinopathyDataset
train_data = RetinopathyDataset(csv_file='/home/v1xjian2/BDL/Bayesian_DL/blindness_detection/binarised_data.csv', image_folder='/home/v1xjian2/BDL/Bayesian_DL/datasets/aptos2019-blindness-detection/train_images')
from torch.utils.data.sampler import SubsetRandomSampler
train_idx = list(range(2))
import torch
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=SubsetRandomSampler(train_idx))
"""


In [ ]:
'''
for a in train_loader:
    print()
    print(a['image'])
'''

In [ ]:
# train_idx

In [ ]:
def evaluation_function(pred_list, output_softmax_list, target_list, model_disag_score_list, pred_entropy_list, thresholds):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
    '''
    Args:
        pred_list: List of predicted class for test samples.
        output_softmax_list: List of (ensembled) softmax predictions for test samples, which are averaged from n_bma_samples.
        target_list: List of labels of test samples.
        model_disag_score_list: List of model disagreement scores.
        pred_entropy_list: List of entropies for predictive distributions.
        thresholds: List of threshold we want to use in uncertainty guided prediction.

    Returns:
        All metrices we want, stored in a dirctionary.
        The metrices are:   
            1. Test accuracy
            2. Precision
            3. Recall
            4. AU-ROC
            5. AU-PRC
            6. F1 score
        These metrices are tested for (pure) prediction and for (different level of) uncertainty guided prediction.
    '''
    # Sort, small -> large
    pred_entropy_sorted_indices = [pred_entropy_list.index(x) for x in sorted(pred_entropy_list)]
    model_disag_score_indices = [model_disag_score_list.index(x) for x in sorted(model_disag_score_list)]

    if 1 not in thresholds:
        thresholds += [1]

    metric_dirct = {}
    for threshold in thresholds:
        index =  int(np.floor(len(pred_list) * threshold))
        curr_accuracy = accuracy_score(target_list[:index], pred_list[:index])
        curr_precision = precision_score(target_list[:index], pred_list[:index])
        curr_recall = recall_score(target_list[:index], pred_list[:index])
        curr_f1 = f1_score(target_list[:index], pred_list[:index])
        curr_auroc = roc_auc_score(target_list[:index], output_softmax_list[:index])
        curr_auprc = average_precision_score(target_list[:index], output_softmax_list[:index])

        jj = int(np.floor(threshold * 100))
        exec(f'metric_dirct["accuracy_{jj}"] = curr_accuracy')
        exec(f'metric_dirct["precision_{jj}"] = curr_precision')
        exec(f'metric_dirct["recall_{jj}"] = curr_recall')
        exec(f'metric_dirct["f1_{jj}"] = curr_f1')
        exec(f'metric_dirct["auroc_{jj}"] = curr_auroc')
        exec(f'metric_dirct["auprc_{jj}"] = curr_auprc')

    return metric_dirct